Импортируем необходимые библиотеки

In [16]:
import math
import os
import pylab
import csv
import pymorphy2

Реализуем метод, для удаления подстрок из черного списка и замены слов на их начальную форму 

In [17]:
def fix_text(strings):
    functors_pos = {'INTJ', 'PRCL', 'CONJ', 'PREP', 'NUMB'}
    black_list = [',', '.', '!','?','-',';','\'','/','(',')','[',']','{','}','–', '+']
    strings_list = []
    morph = pymorphy2.MorphAnalyzer()
    # цикл для удаления подстрок из black_list 
    for i in range(len(strings)):
        for j in range(len(black_list)):
            strings[i] = strings[i].replace(black_list[j],'')
        temp = strings[i].lower().split();
        strings_list.append([])
        # цикл для замены слова на его начальную форму 
        for j in range(len(temp)):
            p = morph.parse(temp[j])
            if len(p)>0 and p[0].tag.POS not in functors_pos:
                strings_list[len(strings_list)-1].append(p[0].normal_form)
            else:
                strings_list[len(strings_list)-1].append(temp[j])
    return strings_list

In [18]:
def csv_dict_writer(path, fieldnames, data):
    """
    Writes a CSV file using DictWriter
    """
    with open(path, "w", newline='') as out_file:
        writer = csv.DictWriter(out_file, delimiter=';', fieldnames=fieldnames)
        writer.writeheader()
        for row in data:
            writer.writerow(row)

Получим список текстов из библиотеки текстов

Считаем содержимое выбранного документа

In [19]:
dirs = os.listdir("dictionary")
arr = []
for k in range(len(dirs)):
    files = os.listdir("dictionary/"+dirs[k])
    collection = ""
    for j in range(len(files)):
        with open('dictionary/'+dirs[k]+"/"+files[j]) as f:
            line = f.readline()
            
            while line:
                collection += line + " "
                line = f.readline()
    arr.append(collection)
#ind = int(input())
#with open('dictionary/'+files[ind]) as f:
#    arr = f.readlines()

In [20]:
n = len(arr)
# список, содержащий списки слов каждого предложения
letter = fix_text(arr)
# множество всех слов
word_set = {}
for i in range(n):
    word_set = set(word_set).union(set(letter[i]))
    
word_dict_arr = [dict.fromkeys(word_set, 0) for i in range(n)]
# считаем количество вхождения слов в кажом предложении
for i in range(n):
    for word in letter[i]:
        word_dict_arr[i][word] += 1

# вычисляем tf
def compute_tf(word_dict, l):
    tf = {}
    sum_nk = len(l)
    for word, count in word_dict.items():
        tf[word] = count / float(sum_nk)
    return tf

# вычисляем tf для каждого предложения
tf_arr = [compute_tf(word_dict_arr[i], letter[i]) for i in range(n)]

# вычисляем idf
def compute_idf(strings_list):
    n = len(strings_list)
    idf = dict.fromkeys(strings_list[0].keys(), 0)
    for l in strings_list:
        for word, count in l.items():
            if count > 0:
                idf[word] += 1

    for word, v in idf.items():
        idf[word] = math.log(n / float(v), 10)
    return idf

# вычисляем idf для каждого предложения
idf = compute_idf(word_dict_arr)
# вычисляем tf и idf
def compute_tf_idf(tf, idf):
    tf_idf = dict.fromkeys(tf.keys(), 0)
    for word, v in tf.items():
        tf_idf[word] = tf[word] * idf[word]
    return tf_idf

# вычисляем tf и idf для каждого предложения
tf_idf_arr = [compute_tf_idf(tf_arr[i], idf) for i in range(n)]
# вывод результата
# for i in range(n):
#     print(tf_idf_arr[i])
my_list = []
for word in word_set:
    my_list.append({" ": word})
    for i in range(len(dirs)):
        dirname = dirs[i]
        my_list[len(my_list)-1][dirname] = tf_idf_arr[i][word]
    
dirs.insert(0," ")
csv_dict_writer("tf_idf.csv", dirs, my_list)